In [1]:
import numpy as np
import cv2
from skimage import io
import os
from tqdm import tqdm

In [2]:
def flip_vertical(image_height, point):
    """
    Cette fonction prend en entrée les coordonnées d'un point dans une image,
    ainsi que la hauteur de l'image, et renvoie les coordonnées du même point
    si l'image est retournée verticalement.
    """
    x, y = point  # sépare les coordonnées x et y du point
    x_flipped = image_height - x  # calcule la nouvelle coordonnée y flippée
    return (x_flipped, y)  # renvoie les coordonnées flippées du point


def add_random_shadow(img, w_low=0.6, w_high=0.85):
    """
    Overlays supplied image with a random shadow polygon
    The weight range (i.e. darkness) of the shadow can be configured via the interval [w_low, w_high)
    """
    cols, rows = (img.shape[0], img.shape[1])
    
    top_y = np.random.random_sample() * rows
    bottom_y = np.random.random_sample() * rows
    bottom_y_right = bottom_y + 5 * np.random.random_sample() * (rows - bottom_y)
    top_y_right = top_y + 5 *np.random.random_sample() * (rows - top_y)
    if np.random.random_sample() <= 0.5:
        bottom_y_right = bottom_y - np.random.random_sample() * (bottom_y)
        top_y_right = top_y - np.random.random_sample() * (top_y)    
    poly = np.asarray([[ [top_y,0], [bottom_y, cols], [bottom_y_right, cols], [top_y_right,0]]], dtype=np.int32)
        
    mask_weight = np.random.uniform(w_low, w_high)
    origin_weight = 1 - mask_weight
    
    mask = np.copy(img).astype(np.int32)
    cv2.fillPoly(mask, poly, (0, 0, 0))
    #masked_image = cv2.bitwise_and(img, mask)
    
    return cv2.addWeighted(img.astype(np.int32), origin_weight, mask, mask_weight, 0).astype(np.uint8)


def add_light_ray(img, brightness_factor=2, ray_width=100):
    """
    Adds a light ray to the input image by increasing the brightness along a vertical line
    spanning from the top to the bottom of the image
    """
    # Get image dimensions
    rows, cols, channels = img.shape
    
    # Choose random center point for the light ray
    center_x = np.random.uniform(0.4, 0.6) * cols
    angle = np.random.randint(80, 100)
    radian = np.deg2rad(angle)
    length = rows
    
    # Calculate start and end points of the light ray
    start = (int(center_x), 0)
    #print(start)
    #end = (int(center_x - length*np.tan(radian)), rows)
    end = np.random.random_sample() * rows
    end = (int(end), rows)
    #print(end)
    
    # Create a mask for the light ray
    mask = np.zeros((rows, cols))
    cv2.line(mask, start, end, 1, thickness=ray_width)
    
    # Increase brightness along the light ray
    img_copy = img.astype(np.float64)
    img_copy[mask==1] *= brightness_factor
    img_copy = np.clip(img_copy, 0, 255)
    img_copy = img_copy.astype(np.uint8)
    
    return img_copy


def augment_image(image, apex):
    # détermine aléatoirement le nombre d'augmentations à appliquer
    num_augmentations = np.random.randint(1, 5)
    #print(num_augmentations)
    # applique les augmentations sélectionnées
    for i in range(num_augmentations):
        # détermine aléatoirement le type d'augmentation à appliquer
        augmentation_type = np.random.randint(1, 4)

        #if augmentation_type == 0:
            # flip vertical
        #    image = np.fliplr(image)
        #    apex = flip_vertical(image.shape[0], apex)
        if augmentation_type == 1:
            # brightness random
            brightness_factor = np.random.uniform(0.2, 1.8)
            image = cv2.convertScaleAbs(image, alpha=brightness_factor, beta=0)
        elif augmentation_type == 2:
            # contrast random
            contrast_factor = np.random.uniform(0.2, 1.8)
            image = cv2.convertScaleAbs(image, alpha=contrast_factor, beta=128*(1-contrast_factor))
            
        elif augmentation_type == 3:
            temp = np.random.randint(0,2)
            if temp == 0:
                image = add_random_shadow(image)
            else:
                fact = np.random.uniform(1,2.2)
                w = np.random.randint(75,175)
                image = add_light_ray(image, fact, w)
            
    return image, apex

In [3]:
images = os.listdir("../data/datasets/final_apex/train/apex/")
images = ["../data/datasets/final_apex/train/apex/"+item for item in images]
output_folder = "../data/datasets/final_apex/aug_train/"

cmp = 0
for image in tqdm(images):
    temp = image.split('/')[-1][:10].split('_')
    coord = (int(temp[0]), int(temp[1]))
    img = io.imread(image)
    
    io.imsave(output_folder+image.split('apex/')[-1], img)
    cmp += 1
    
    new_img, new_coord = augment_image(img, coord)
    new_name = output_folder + str(new_coord[0]) + "_" + str(new_coord[1])+"_augmented_nb_"+str(cmp)+".jpg"
    cmp += 1
    io.imsave(new_name, new_img)
    
    new_coord = flip_vertical(img.shape[0], coord)
    new_img, _ = augment_image(img, new_coord)
    new_name = output_folder + str(new_coord[0]) + "_" + str(new_coord[1])+"_augmented_nb_"+str(cmp)+".jpg"
    cmp += 1
    io.imsave(new_name, new_img)

  3%|█                                        | 21/798 [00:00<00:07, 105.50it/s]/tmp/ipykernel_15737/2629193629.py:17: UserWarning: ../data/datasets/final_apex/aug_train/41_122_augmented_nb_73.jpg is a low contrast image
  io.imsave(new_name, new_img)
  9%|███▌                                     | 69/798 [00:00<00:06, 112.41it/s]/tmp/ipykernel_15737/2629193629.py:17: UserWarning: ../data/datasets/final_apex/aug_train/240_151_augmented_nb_211.jpg is a low contrast image
  io.imsave(new_name, new_img)
 10%|████▏                                    | 82/798 [00:00<00:06, 116.46it/s]/tmp/ipykernel_15737/2629193629.py:17: UserWarning: ../data/datasets/final_apex/aug_train/9_166_augmented_nb_259.jpg is a low contrast image
  io.imsave(new_name, new_img)
 17%|██████▌                                 | 132/798 [00:01<00:05, 117.59it/s]/tmp/ipykernel_15737/2629193629.py:23: UserWarning: ../data/datasets/final_apex/aug_train/109_120_augmented_nb_416.jpg is a low contrast image
  io.imsave(new_nam

 82%|████████████████████████████████▋       | 652/798 [00:05<00:01, 118.57it/s]/tmp/ipykernel_15737/2629193629.py:17: UserWarning: ../data/datasets/final_apex/aug_train/135_130_augmented_nb_1984.jpg is a low contrast image
  io.imsave(new_name, new_img)
 83%|█████████████████████████████████▎      | 665/798 [00:05<00:01, 119.03it/s]/tmp/ipykernel_15737/2629193629.py:17: UserWarning: ../data/datasets/final_apex/aug_train/237_158_augmented_nb_2026.jpg is a low contrast image
  io.imsave(new_name, new_img)
 86%|██████████████████████████████████▌     | 689/798 [00:06<00:00, 112.92it/s]/tmp/ipykernel_15737/2629193629.py:23: UserWarning: ../data/datasets/final_apex/aug_train/126_118_augmented_nb_2087.jpg is a low contrast image
  io.imsave(new_name, new_img)
 92%|████████████████████████████████████▉   | 738/798 [00:06<00:00, 115.66it/s]/tmp/ipykernel_15737/2629193629.py:17: UserWarning: ../data/datasets/final_apex/aug_train/86_124_augmented_nb_2227.jpg is a low contrast image
  io.imsave(